In [1]:
import numpy as np

In [10]:
def convForward(A_prev,hyperparameters):
    (m,n_H,n_W,n_C)=A_prev.shape
    (f,f,n_F)=W.shape
    W=hyperparameters('W')
    b=hyperparameters('b')
    f=hyperparameters('f')
    s=hyperparameters('s')
    
    Z=np.zeros(m,((n_H-f)/s)+1,((n_W-f)/s)+1,n_F)
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_F):
                    a_slice=A_prev[i,h:h+f,w:w+f,:]
                    Z[i,h,w,c]=conv(a_slice,W,b)
    A=relu(Z)
    cache=(A)

In [11]:
def conv(a_slice,w,b):
    return np.sum(np.multiply(a_slice,w))+b

In [4]:
def relu(Z):
    return np.max(Z,0)

In [15]:
def sigmoid(z):
    return(1/(1+np.exp(-z)))

In [7]:
def forwardPool(A_prev,hyperparameters,mode='max'):
    f=hyperparameters('f')
    s=hyperparameters('s')
    
    (m,n_H,n_W,n_C)=A_prev.shape
    A=np.zeros(m,((n_H-f)/s)+1,((n_W-f)/s)+1,n_C)
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    a_slice=A_prev[i,h:h+f,w:w+f,c]
                    if(mode=='max'):
                        A[i,h,w,c]=np.max(a_slice)
                    elif(mode=='average'):
                        A[i,h,w,c]=np.average(a_slice)
    return A                    

In [9]:
def poolBackward(dA,cache,hyperparameters,mode='max'):
    A_prev=cache('A_prev')
    s=hyperparameters['s']
    f=hyperparameters['f']
    
    (m,n_H,n_W,n_C)=A_prev.shape
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    a_slice=A_prev[i,h:h+f,w:w+f,c]
                    if(mode=='max'):
                        a_slice-=(a_slice==np.max(a_slice))*dA[i,h,w,c]
                    elif(mode=='average'):
                        a_slice-=dA[i,h,w,c]/(f*f)
    return A_prev                   

In [13]:
def convBackward(dZ,cache):
    (A_prev,W,b)=cache
    (m,n_H,n_W,n_C)=A_prev.shape
    
    dA_prev=np.zeros(m,n_H,n_W,n_C)
    dW=np.zeros(W.shape)
    for i in range(m):
        for h in range(n_H):
            for w in range(n_W):
                for c in range(n_C):
                    a_slice=A_prev[i,h:h+f,w:w+f,c]
                    dA_prev[i,h:h+f,w:w+f,:]-=W*dz[i,h,w,c]
                    dW-=A_prev[i,h:h+f,w:w+f,:]*dz[i,h,w,c]
    db=np.sum(dz)  
    return (dA_prev,dW,db)

In [14]:
def FCForward(A_prev,hyperparameters):
    W=hyperparameters('W')
    b=hyperparameters('b')
    Z=A_prev.dot(W)+b
    A=relu(Z)
    cache=A
    return A

In [17]:
def FCBackward(dA,cache,activation='relu'):
    (A_prev,W,b)=cache
    Z=A_prev.dot(W)+b
    if(activation=='relu'):        
        dW=(Z[Z>0]*dA).T.dot(A_prev.T)
        dA_prev=(dA*Z[Z>0]).dot(W.T)
    elif(activation=='sigmoid'):
        dW=(dA*sigmoid(Z)*(1-sigmoid(Z))).T.dot(A_prev.T)
        dA_prev=(dA*sigmoid(Z)*(1-sigmoid(Z))).dot(W.T)
    return dA_prev,dW    